In [1]:
using WordNet
using AdaGram
using AdaGramCompat

using CorpusLoaders
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using LightXML
using JLD

In [2]:
const WN_PATH="data/corpora/WordNet-2.1/"
#WN_PATH3 = "/usr/share/nltk_data/corpora/wordnet/"
db = DB(WN_PATH)

WordNet.DB

In [ ]:
#am = AdaGramCompat.AdaGramModel(load_model("models/adagram/v1_d100.adagram_model")...)
am = AdaGramCompat.AdaGramModel(load_model("models/adagram/more_senses.adagram_model")...)

In [ ]:
#ee = restore("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld");
ee = load("models/ss/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.jld", "ee")

#ee = restore("models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_50__m170000000.model");

In [ ]:
CorpusLoaders.punctuation_space_tokenize

In [ ]:
reload(CorpusLoaders)

In [ ]:
using CorpusLoaders.Semeval2007t7

challenges = load_challenges_semeval2007t7("data/corpora/wsd/semeval2007_t7/test/eng-coarse-all-words.xml");
solutions = load_solutions_semeval2007t7("data/corpora/wsd/semeval2007_t7/key/dataset21.test.key");

In [ ]:
only_of_pos(data, pos) = filter(d->d.pos==pos, data)

In [ ]:
function sense_frequency(ss::Synset, lem::Lemma)
    lem_count = lem.tagsense_count
    #lem_count+0.1(sum(values(ss.word_counts))-lem_count) #Do not use Synset's other counts. It owrks out worse
    lem_count
end

In [ ]:
typemax(Int64)

In [45]:
const AdaGram_lib2 = AdaGramCompat.AdaGram_lib
function Query.logprob_of_context(am::AdaGramModel, context, input::AbstractVector{Float32}; skip_oov=false, normalise_over_length=false)
    total_lprob=zero(Float32)
    context_length = 0
    for context_word in context
        skip_oov && !haskey(am.dict.word2id, context_word) && continue
        context_length+=1
        context_word_id = am.dict.word2id[context_word]
        context_word_path = view(am.vm.path, :, context_word_id)
        context_word_code = view(am.vm.code, :, context_word_id)

        word_lprob = ccall((:skip_gram, AdaGram_lib2), Float32,
            (Ptr{Float32}, Ptr{Float32}, Clonglong, Ptr{Int32}, Ptr{Int8},        Clonglong),
            input, am.vm.Out,  size(am.vm.In, 1), context_word_path, context_word_code, size(am.vm.code, 1))
        total_lprob+=word_lprob
    end
    if normalise_over_length
        total_lprob/=context_length #This is equivlent to taking the context_length-th root in nonlog domain. Which makes sense.
    end
    total_lprob
end



In [46]:
function all_word_sense_vectors(ee::WordSenseEmbedding, word)
    get(ee.embedding, word, Vector{Float32}[])
end

function all_word_sense_vectors(am::AdaGramCompat.AdaGramModel, word)
    if haskey(am.dict.word2id, word)
        wsv_mat = word_sense_vectors(am, word)
        [view(wsv_mat,:,ii) for ii in 1:size(wsv_mat,2)]
    else
        Vector{Float32}[]
    end
end



all_word_sense_vectors (generic function with 2 methods)

In [47]:
function normal_probs(logprobs::Vector)
    ret = copy(logprobs)
    max_lp = maximum(logprobs)
    ret.-=max_lp #Bring closer to zero
    map!(exp,ret)
    denom = sum(ret)
    ret./=denom
    ret
end
function weighted_average(logprobs, embeddings)
    ret = zeros(first(embeddings))
    for (weight, embedding) in zip(normal_probs(logprobs), embeddings)
        ret.+= weight.*embedding
    end
    ret
end


function synthesize_embedding(ee,context, word_or_phrase)
    words = split(word_or_phrase, " ")
    wvs = vcat((all_word_sense_vectors(ee,w) for w in words)...)
    if length(wvs) == 0
            throw(KeyError("None of $words have embeddings"))
    end
    logprobs = [Query.logprob_of_context(ee, context,wv; skip_oov=true, normalise_over_length=true) for wv in wvs]
    weighted_average(logprobs, wvs)
end


all_identical(col) = length(col)==1 || !any(x->x!=col[1],col[2:end])

lexically_informed_embeddings(wn::DB, ee, lemma_word, pos) = lexically_informed_embeddings(db,ee, lemma_word, lemma_word, pos)
function lexically_informed_embeddings(wn::DB, ee, word,lemma_word, pos)
    
    lemma = db[pos, lemma_word]
    target_synsets::Vector{Synset} = synsets(db, lemma)
         
    embeddings = Vector{Vector{Float32}}(length(target_synsets))
    for (ii,synset) in enumerate(target_synsets)
        context::Vector{SubString{String}} = collect(filter!(w->w!=word, punctuation_space_tokenize(synset.gloss)))
        embeddings[ii] = synthesize_embedding(ee,context, word)
    end
        
    
    if all_identical(embeddings)
        #Use MCWS
        score, index = findmax(sense_frequency(ss,lemma) for ss in target_synsets)
        target_synsets=[target_synsets[index]] # Throw out others
        embeddings=[embeddings[1]]
    end
        
    embeddings,target_synsets,lemma
end

lexically_informed_embeddings (generic function with 2 methods)

In [48]:
function supervised_wsd(challenge, ee, wn::DB)
    try
        embeddings,target_synsets,lemma = lexically_informed_embeddings(wn,ee,
                                                            challenge.word, 
                                                            challenge.lemma, challenge.pos)
        #sense_index = Query.WSD(ee, embeddings, challenge.context; skip_oov=true)
        logprobs_of_context = [logprob_of_context(ee,challenge.context, wv; skip_oov=true, normalise_over_length=true) 
                                    for wv in embeddings] 
      
        sense_index= indmax(logprobs_of_context)

        
        synset = target_synsets[sense_index]
        sk = sensekey(wn, synset, lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end

function most_frequent_sense(challenge, wn::DB)
    try
        lemma = wn[challenge.pos, challenge.lemma]
        target_synsets::Vector{Synset} = synsets(db, lemma)
        
        sense_freqs =  Float32[sense_frequency(ss,lemma) for ss in target_synsets]      
        sense_index= indmax(sense_freqs)
        synset = target_synsets[sense_index]
        
        sk = sensekey(wn, synset,  lemma)
        Nullable(sk)
    catch ex
        isa(ex, KeyError) || rethrow(ex)
        Nullable{String}()
    end
end



most_frequent_sense (generic function with 1 method)

In [49]:
function evalute_on_wsd_challenge(challenges, solutions, method)
    correct = 0
    incorrect = 0
    notattempted = 0
    for (challenge, ground_solution) in zip(challenges, solutions)
        assert(challenge.id == ground_solution.id)
        output_sense = method(challenge)
        if isnull(output_sense)
            notattempted+=1
        else
            if get(output_sense) ∈ ground_solution.solutions
                correct+=1
            else
                incorrect+=1
            end
        end
    end
    precision = correct/(correct+incorrect)
    recall = correct/(correct+incorrect+notattempted)
    f1 = (2*precision*recall) / (precision+recall)
    return precision, recall, f1
end
    
    
function evalute_on_wsd_challenge(challenges, solutions, ee, wn::DB)
    method = challenge -> supervised_wsd(challenge, ee, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end


function evalute_on_wsd_challenge_MFS(challenges, solutions, wn::DB)
    method = challenge -> most_frequent_sense(challenge, wn)
    evalute_on_wsd_challenge(challenges, solutions, method)
end

evalute_on_wsd_challenge_MFS (generic function with 1 method)

In [56]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, am, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), am, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), am, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), am, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), am, db))

overall: 		(0.6326710816777041,0.6315557514323491,0.6321129245699162)
only nouns  : 		(0.6504065040650406,0.6498194945848376,0.6501128668171559)
only verbs  : 		(0.576271186440678,0.5752961082910322,0.5757832345469941)
only adjecti: 		(0.6260387811634349,0.6243093922651933,0.6251728907330568)
only adverbs: 		(0.7101449275362319,0.7067307692307693,0.708433734939759)


In [42]:
?ccall

search: ccall sense_counts_callback HierarchicalSoftmaxNode AbstractChannel



```
ccall((symbol, library) or function_pointer, ReturnType, (ArgumentType1, ...), ArgumentValue1, ...)
```

Call function in C-exported shared library, specified by `(function name, library)` tuple, where each component is a string or symbol.

Note that the argument type tuple must be a literal tuple, and not a tuple-valued variable or expression. Alternatively, `ccall` may also be used to call a function pointer, such as one returned by `dlsym`.

Each `ArgumentValue` to the `ccall` will be converted to the corresponding `ArgumentType`, by automatic insertion of calls to `unsafe_convert(ArgumentType, cconvert(ArgumentType, ArgumentValue))`. (See also the documentation for each of these functions for further details.) In most cases, this simply results in a call to `convert(ArgumentType, ArgumentValue)`.


In [51]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, ee, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), ee, db))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), ee, db))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), ee, db))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), ee, db))

overall: 		(0.6486486486486487,0.6346408109299251,0.6415682780129205)
only nouns  : 		(0.6593001841620626,0.6462093862815884,0.6526891522333637)
only verbs  : 		(0.5785837651122625,0.5668358714043993,0.5726495726495726)
only adjecti: 		(0.6762177650429799,0.6519337016574586,0.6638537271448665)
only adverbs: 		(0.7427184466019418,0.7355769230769231,0.7391304347826086)


In [52]:
println("overall: \t\t", evalute_on_wsd_challenge_MFS(challenges, solutions, db))

println("only nouns  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), db))
println("only verbs  : \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), db))
println("only adjecti: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), db))
println("only adverbs: \t\t", evalute_on_wsd_challenge_MFS(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), db))


overall: 		(0.7783164389598942,0.7783164389598942,0.7783164389598942)
only nouns  : 		(0.7653429602888087,0.7653429602888087,0.7653429602888087)
only verbs  : 		(0.7529610829103215,0.7529610829103215,0.7529610829103215)
only adjecti: 		(0.8038674033149171,0.8038674033149171,0.8038674033149171)
only adverbs: 		(0.875,0.875,0.875)


In [ ]:
#Zero shot WSI for 1 shot WSD
using Training

In [ ]:
rr = FixedWordSenseEmbedding(ee.dimension, random_inited, huffman_tree; initial_nsenses=1)
rr.distribution = ee.distribution
rr.codebook = ee.codebook
rr.classification_tree = ee.classification_tree
Training.initialize_embedding(rr)

In [ ]:
rr.initial_nsenses=100
Training.initialize_embedding(rr)

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, rr, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), rr, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), rr, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), rr, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), rr, db, sense_key_map))


In [ ]:
hh = deepcopy(rr)
for word in keys(rr.embedding)
    append!(hh.embedding[word], ee.embedding[word])
end

In [ ]:
println("overall: \t\t", evalute_on_wsd_challenge(challenges, solutions, hh, db, sense_key_map))

println("only nouns  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'n'), only_of_pos(solutions,'n'), hh, db, sense_key_map))
println("only verbs  : \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'v'), only_of_pos(solutions,'v'), hh, db, sense_key_map))
println("only adjecti: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'a'), only_of_pos(solutions,'a'), hh, db, sense_key_map))
println("only adverbs: \t\t", evalute_on_wsd_challenge(only_of_pos(challenges,'r'), only_of_pos(solutions,'r'), hh, db, sense_key_map))

In [ ]:
using JLD